In [37]:
from riotwatcher import LolWatcher, ApiError
from riot_api import key
import pandas as pd
import numpy as np
from datetime import datetime
import json
from leagueDataParser import parseLeagueData
from leagueDataCleaner import cleanLeagueData

lol = LolWatcher(key)

In [3]:
data = pd.read_csv('data.csv').iloc[:,1:]
data

,Pseudo,Région / TagLine,Date de naissance,Sexe,Département de résidence (numéro) actuel,Avez vous un job / Etes vous étudiant ?,Statut familial,Situation domicile,Achetez vous du contenu de jeu ?,"Niveau économique (1 = plus bas, 3 = plus haut)",Aimez vous le travail d'équipe ?,Savez vous jouer d'un instrument ?,Pratiquez vous du sport ?
0,Katiounette,EUW,01/10/2001,Femme,91,Oui,En couple,Je vis avec d'autres personnes,Rarement,2,Oui,Non,Parfois
1,Jazzon18,EUW,18/10/2001,Homme,33,Oui,En couple,Je vis seul,Rarement,2,Oui,Non,En amateur
2,Choc Epic,EUW,25/01/2001,Homme,33,Non,En couple,Je vis avec d'autres personnes,Rarement,1,Oui,Oui,Parfois
3,OhSSSans17,EUW,01/01/2001,Homme,91,Oui,En couple,Je vis avec d'autres personnes,Non,3,Oui,Oui,Non


In [4]:
targets = pd.DataFrame(columns=['pseudo', 'region', 'age', 'sex', 'department', 'job', 'relationship', 'live_with_others', 'buy_content', 'economic', 'love_team_work',  'play_instrument', 'sport'])
targets

,pseudo,region,age,sex,department,job,relationship,live_with_others,buy_content,economic,love_team_work,play_instrument,sport


In [5]:
targets['pseudo'] = data['Pseudo']

In [6]:
targets['region'] = data['Région / TagLine']
targets

,pseudo,region,age,sex,department,job,relationship,live_with_others,buy_content,economic,love_team_work,play_instrument,sport
0,Katiounette,EUW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Jazzon18,EUW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Choc Epic,EUW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OhSSSans17,EUW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
targets['age'] = np.floor((pd.to_datetime(pd.Timestamp.now()) - pd.to_datetime(data['Date de naissance'], format = "%d/%m/%Y")).dt.days / 365.25)

In [43]:
targets

,pseudo,region,age,sex,department,job,relationship,live_with_others,buy_content,economic,love_team_work,play_instrument,sport
0,Katiounette,EUW,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Jazzon18,EUW,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Choc Epic,EUW,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OhSSSans17,EUW,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
targets['sex'] = np.where(data['Sexe'] == 'Femme', 0, 1 )
targets

,pseudo,region,age,sex,department,job,relationship,live_with_others,buy_content,economic,love_team_work,play_instrument,sport
0,Katiounette,EUW,21.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Jazzon18,EUW,21.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Choc Epic,EUW,22.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OhSSSans17,EUW,22.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
targets['department'] = data['Département de résidence (numéro) actuel']

In [48]:
targets

,pseudo,region,age,sex,department,job,relationship,live_with_others,buy_content,economic,love_team_work,play_instrument,sport
0,Katiounette,EUW,21.0,0,91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Jazzon18,EUW,21.0,1,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Choc Epic,EUW,22.0,1,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OhSSSans17,EUW,22.0,1,91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
targets['job'] = np.where(data['Avez vous un job / Etes vous étudiant ?'] == 'Oui', 1, 0)
targets

,pseudo,region,age,sex,department,job,relationship,live_with_others,buy_content,economic,love_team_work,play_instrument,sport
0,Katiounette,EUW,21.0,0,91,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Jazzon18,EUW,21.0,1,33,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Choc Epic,EUW,22.0,1,33,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OhSSSans17,EUW,22.0,1,91,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
targets['relationship'] = np.where(data['Statut familial'] == 'En couple', 1, 0)
targets

,pseudo,region,age,sex,department,job,relationship,live_with_others,buy_content,economic,love_team_work,play_instrument,sport
0,Katiounette,EUW,21.0,0,91,1,1,NaN,NaN,NaN,NaN,NaN,NaN
1,Jazzon18,EUW,21.0,1,33,1,1,NaN,NaN,NaN,NaN,NaN,NaN
2,Choc Epic,EUW,22.0,1,33,0,1,NaN,NaN,NaN,NaN,NaN,NaN
3,OhSSSans17,EUW,22.0,1,91,1,1,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
targets['live_with_others'] = np.where(data['Situation domicile'] == "Je vis avec d'autres personnes", 1, 0)
targets

,pseudo,region,age,sex,department,job,relationship,live_with_others,buy_content,economic,love_team_work,play_instrument,sport
0,Katiounette,EUW,21.0,0,91,1,1,1,NaN,NaN,NaN,NaN,NaN
1,Jazzon18,EUW,21.0,1,33,1,1,0,NaN,NaN,NaN,NaN,NaN
2,Choc Epic,EUW,22.0,1,33,0,1,1,NaN,NaN,NaN,NaN,NaN
3,OhSSSans17,EUW,22.0,1,91,1,1,1,NaN,NaN,NaN,NaN,NaN


In [ ]:
targets['buy_content'] = 